# Module 6 - Fine tuning ResNet toward plankton data

We can also exploit the generality of low level features in neural networks to *fine tune* a generic object classifer. This is akin to teaching a person without prior oceanographic experience how to recognize different types of fish. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

## General training model 

## Data transforms 

### Training versus validation data

## Reset final fully connected layer

In [ ]:
# define the model to use
model_conv = models.resnet34(pretrained=True)

# get the number of features that are input to the fully connected layer
num_ftrs = model_conv.fc.in_features

# reset the fully connect layer
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

## Train

In [ ]:
# how many epochs for this? not very many

## Evaluate

# Compare